In [3]:
import os
import json
import time
import torch
import random
from common import *
from vllm import LLM, SamplingParams
from transformers import (AutoModelForCausalLM, AutoTokenizer,
                          PreTrainedTokenizerBase)


def create_filtered_dataset(dataset: list, tokenzier:str) -> list:
    tokenzier = AutoTokenizer.from_pretrained(tokenzier, trust_remote_code=LLM_ARGS['trust_remote_code'])

    filtered_dataset = list()
    
    for data in dataset:
        prompt_token_ids = tokenzier(data).input_ids
        if len(prompt_token_ids) < 4 or len(prompt_token_ids) > 1024:
            continue
        filtered_dataset.append(data)
        if len(filtered_dataset) > 500:
            break
    
    return filtered_dataset

def load_dataset(dataset_path: str)-> list:
    
    print(f"Loading datasetfrom {dataset_path}...")
    with open(dataset_path) as f:
        dataset = json.load(f)
    
    # Filter out the conversations with less than 2 turns.
    dataset = [data for data in dataset if len(data["conversations"]) >= 2]
    # Only keep the first two turns of each conversation.
    dataset = [(data["conversations"][0]["value"],
                data["conversations"][1]["value"]) for data in dataset]

    random.shuffle(dataset)
    
    prompts_dataset = [data[0] for data in dataset]
    
    print(f"Done loading dataset.")
    model='google/gemma-2b'
    return create_filtered_dataset(prompts_dataset,model)

if __name__ == '__main__':
    dataset_path = '/home/mcw/thrisha/data/ShareGPT_V3_unfiltered_cleaned_split.json'

    dataset=load_dataset(dataset_path=dataset_path)
    print(dataset[0:5])

Loading datasetfrom /home/mcw/thrisha/data/ShareGPT_V3_unfiltered_cleaned_split.json...
Done loading dataset.
['I want you to act as a headline generator for a news article. Your task is to come up with catchy and attention-grabbing headlines that accurately summarize the content of the article. Keep in mind the tone of the article and make sure the headline is engaging and informative. Your response should only be one headline for each article description I provide. Do not include any subheadings or additional information.', 'What did Heisenberg explain on resonance theory ?', "As a software engineer, some examples of goals and priorities you can have include:\n\n1. Improving your technical skills and knowledge by learning new programming languages, frameworks, and technologies.\n2. Building and delivering high-quality, reliable, and maintainable software that meets customer needs and exceeds expectations.\n3. Developing a deep understanding of the problem domain and user needs to cre

In [5]:
len(dataset)

501

In [6]:
def load_prompts(dataset: list, batch_size:int) -> list:
    return random.sample(dataset, batch_size)
    
d=load_prompts(dataset,8)
d

["can you please explain vector search to me like I'm 5?",
 "Here's a possible response:\n\nRéponse:\n\nLors de l'utilisation de SQL, il est important de comprendre les basiques pour pouvoir tirer le meilleur parti de ce langage de programmation de base de données. Voici une fiche rappel organisée de manière claire pour vous aider.\n\n1. Syntaxe de base:\nSQL est composé de commandes qui sont utilisées pour accéder et manipuler des données dans une base de données. Les commandes les plus courantes sont SELECT, INSERT, UPDATE et DELETE.\n2. Les requêtes SELECT:\nLa requête SELECT est l'une des commandes les plus courantes en SQL. Elle est utilisée pour récupérer des données dans une base de données. Pour utiliser SELECT, il faut spécifier les colonnes que vous souhaitez récupérer et la table à partir de laquelle vous voulez récupérer les données.\n3. Les filtres WHERE:\nLa clause WHERE est utilisée pour filtrer les données dans une requête SELECT. Elle permet de sélectionner des enregis

In [9]:
d

["can you please explain vector search to me like I'm 5?",
 "Here's a possible response:\n\nRéponse:\n\nLors de l'utilisation de SQL, il est important de comprendre les basiques pour pouvoir tirer le meilleur parti de ce langage de programmation de base de données. Voici une fiche rappel organisée de manière claire pour vous aider.\n\n1. Syntaxe de base:\nSQL est composé de commandes qui sont utilisées pour accéder et manipuler des données dans une base de données. Les commandes les plus courantes sont SELECT, INSERT, UPDATE et DELETE.\n2. Les requêtes SELECT:\nLa requête SELECT est l'une des commandes les plus courantes en SQL. Elle est utilisée pour récupérer des données dans une base de données. Pour utiliser SELECT, il faut spécifier les colonnes que vous souhaitez récupérer et la table à partir de laquelle vous voulez récupérer les données.\n3. Les filtres WHERE:\nLa clause WHERE est utilisée pour filtrer les données dans une requête SELECT. Elle permet de sélectionner des enregis

In [ ]:
from huggingface_hub import login
login()